# Context Management in OpenAI Agents SDK

**Context Management** in the OpenAI Agents SDK allows agents to maintain, access, and transfer state information during a conversation or workflow. The SDK uses a special object called RunContextWrapper to manage this context, ensuring that each agent or tool has access to relevant data from the ongoing interaction.

🔹 **Key Concepts**:

- **`RunContextWrapper`**: A wrapper around the current run’s state. It holds inputs, outputs, and metadata for the agent during execution.

- **Persistence Across Steps**: When agents or tools are invoked, the context allows them to “remember” previous turns or actions.

- **Data Sharing**: Enables sharing of information between agents, tools, and guardrails within the same run.

- **Handoff Management**: When one agent delegates a task to another, the context ensures that necessary data (like user input or prior results) is seamlessly passed along.

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")

if not api_key:
    # Prompt the user to enter the API key if not set
    import getpass
    api_key = getpass.getpass("Please enter your OPENAI_API_KEY: ")
    if not api_key:
        raise ValueError("OPENAI_API_KEY variable is not set in the environment variables or provided by user.")

In [ ]:
from dataclasses import dataclass
import random

from agents import Agent, RunContextWrapper, Runner, TResponseInputItem, function_tool

@dataclass
class user_profile:
    id: int
    name: str
    shopping_card: list[str]

@function_tool
async def get_budget(wrapper: RunContextWrapper[user_profile]):
    """
    Get the user's budget using the user id and linked bank account.
    """
    print("Getting the account balance...")
    user_id = wrapper.context.id
    return 100.0

@function_tool
async def search_products(wrapper: RunContextWrapper[user_profile], product: str):
    """
    Search for products in the database.
    """
    print("Searching....")

    #randomly generate prices of the products
    price = random.randint(1, 100)
    return [{"name": product, "price": price}]

@function_tool
async def add_to_cart(wrapper: RunContextWrapper[user_profile], items: list[str]):
    """
    Add items to the user's shopping cart.
    """
    print("Adding items to the cart...")
    wrapper.context.shopping_card.extend(items)


@function_tool
async def checkout(wrapper: RunContextWrapper[user_profile]) -> None:
    """
    Checkout the user's shopping cart.
    """
    print("Checking out...")
    
    # we could take the items from the shopping cart and purchase them using some external API
    # for now, we'll just print a message

    print(f"Successfully purchased items: {wrapper.context.shopping_card}")



shopping_agent = Agent[user_profile](
    name="Shopping Agent",
    instructions=(
        "You are a shopping assistant dedicated to helping the user with their grocery shopping needs."
        "Your primary role is to assist in creating a shopping plan that fits within the user's budget."
        "Start by getting the user's budget using the tool get_budget."
        "Provide suggestions for items if requested, and always aim to keep the total cost within the user's budget."
        "If the user is nearing or exceeding their budget, inform them and suggest alternatives or adjustments to the shopping list."
        "If the user authorizes it, you can purchase the items using the tool purchase_items."
    ),
    tools=[get_budget, search_products, add_to_cart, checkout],
    model="gpt-4o-mini",
)

profile = user_profile(id=4824, name="Ehtesham", shopping_card=[])

print("You are now chatting with the shopping agent. Type 'exit' to quit.")

conversation: list[TResponseInputItem] = []
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    print("You: ", user_input)
    conversation.append({"role": "user", "content": user_input})

    response = await Runner.run(shopping_agent, conversation, context=profile)

    print(f"Shopping Assistant: {response.final_output}")

    conversation = response.to_input_list()  # maintain the conversation history

    

You are now chatting with the shopping agent. Type 'exit' to quit.
You:  hi
Shopping Assistant: Hello! How can I assist you with your grocery shopping today?
You:  i am feeling hungry
Shopping Assistant: Let’s find something delicious for you! Do you have a specific type of food in mind, or would you like some suggestions for snacks or meals? Also, it would help to know your budget for shopping.
You:  how about desi food
Getting the account balance...
Shopping Assistant: Great choice! Desi food has a lot of flavorful options. Here are some popular desi food items to consider:

1. **Biryani** (chicken, mutton, or vegetarian)
2. **Paneer Tikka**
3. **Samosas**
4. **Chole Bhature**
5. **Dal Makhani**
6. **Naan or Roti**
7. **Raita**
8. **Gulab Jamun** for dessert

Your budget is $100. Would you like to focus on a specific dish, or would you like a mix of several items?
You:  Aaloo biryani options
Searching....
Shopping Assistant: I found an option for **Aaloo Biryani** priced at **$4**. W